# User Manual

## Overview

This notebook provides a complete solution for archiving Rizzoma topics and importing them into Notion with high fidelity. It is organized into three distinct, independent modules:

1.  **Acquisition:** Connects to the Rizzoma API to create a complete, self-contained, and permanent `.zip` archive of a topic. This is the only module that requires a connection to Rizzoma.
2.  **Notion Import (From Scratch):** Takes a previously created `.zip` archive from Google Drive and creates a brand new, fully formatted page in Notion.
3.  **Notion Import (Resume):** A specialized workflow to resume a previously failed or interrupted import, appending the remaining content to an existing Notion page.

---

## 1. Initial Setup & Configuration

This is the most critical step. The script will not work without the correct configuration.

### Step 1.1: Obtaining Credentials

You need two secret keys. These should be stored using Colab's "Secrets" feature (accessible via the key icon 🔑 on the left sidebar) for security.

*   **`CONNECT_SID` (for Rizzoma):**
    1.  Open your web browser and log in to [rizzoma.com](https://rizzoma.com).
    2.  Open your browser's **Developer Tools** (usually by pressing F12 or Ctrl+Shift+I).
    3.  Go to the **Application** (or Storage) tab.
    4.  On the left, find the "Cookies" section and select `https://rizzoma.com`.
    5.  Find the cookie named `connect.sid`.
    6.  Copy its **Value**. This is your `CONNECT_SID` secret.

*   **`NOTION_API_KEY` (for Notion):**
    1.  Go to [notion.so/my-integrations](https://www.notion.so/my-integrations).
    2.  Click "**+ New integration**".
    3.  Give it a name (e.g., "Rizzoma Importer") and associate it with your workspace.
    4.  On the next screen, copy the **Internal Integration Token**. This is your `NOTION_API_KEY` secret.
    5.  **Crucially:** Go to the Notion page you want to import into (or its parent) and use the "Share" menu to invite and share the page with the integration you just created.


**How to Store the Secrets:**

1.  In your Colab notebook, click on the **key icon (🔑)** in the left sidebar to open the "Secrets" panel.
2.  Click **"+ Add a new secret"**.
3.  For the **Name**, enter `CONNECT_SID`. For the **Value**, paste the key you will obtain from Rizzoma (instructions below).
4.  Ensure the toggle switch **"Notebook access"** is turned **on**.
5.  Repeat the process: click **"+ Add a new secret"** again, create a secret named `NOTION_API_KEY`, and paste its corresponding value.

The code in this notebook uses `userdata.get('SECRET_NAME')` to securely access these values without them being visible.

### Step 1.2: Configuring the Notebook

In the **"Global Configuration"** cell, you must fill in the following parameters:

*   **`GDRIVE_ARCHIVE_PATH`**: The path to the folder on your Google Drive where the `.zip` archives will be saved.
    *   **How to get it:** In the Colab file browser on the left, navigate to your desired folder inside `drive/MyDrive/`, right-click on it, and select "Copy path".
    *   *Example:* `'/content/drive/MyDrive/Rizzoma Archives'`

*   **`NOTION_PARENT_PAGE_ID`**: The ID of the Notion page under which new topic pages will be created during a fresh import.
    *   **How to get it:** Go to the page in Notion. The URL will look like `.../My-Page-Title-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`. The 32-character string at the end is the ID.
    *   *Example:* `'2a50e93a826b805f831fe41fa47c8bbc'`

*   **`TOPIC_IDS_FOR_ACQUISITION`**: A Python list of Rizzoma topic IDs (as strings) that you want to archive.
    *   **How to get it:** In Rizzoma, the URL of a topic is `.../topic/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx/`. The 32-character string is the ID.
    *   *Example:* `['366d5a79a54be17288daab770d4833d9', '0f475b2af107fc990cb199f00a0f4e89']`

*   **`TOPIC_IDS_FOR_OFFLINE_MODES`**: A list of topic IDs to process for the Notion Import workflows. These must correspond to archives that already exist in your `GDRIVE_ARCHIVE_PATH`.

*   **`PRUNE_OLD_BLIPS`**: A boolean (`True` or `False`). If `True`, any line in Rizzoma containing only the word "old" followed by a blip will be completely ignored during the Notion import, cleaning up old archives.

---

## 2. Core Workflows

Once configured, you can run the following workflows independently.

### Workflow 1: Archiving a New Topic (Acquisition)

This is the first step for any topic. It connects to Rizzoma and creates a permanent, self-contained archive.

*   **How to run:**
    1.  Fill in the `TOPIC_IDS_FOR_ACQUISITION` list in the configuration cell.
    2.  Execute the cell containing the `run_acquisition(...)` function.

*   **How to verify it worked:**
    *   The notebook will log the download progress for each file.
    *   At the end, you will find a new `.zip` file (e.g., `Topic_Title__<topic_id>.zip`) in the `GDRIVE_ARCHIVE_PATH` folder on your Google Drive.
    *   If some files failed to download (e.g., due to a 404 error), a `download_log.md` file will be included inside the archive detailing the missing files. The process will **not** stop for these errors.

### Workflow 2: Importing an Archive to Notion (From Scratch)

This workflow takes a `.zip` archive and creates a brand new page in Notion.

*   **How to run:**
    1.  Ensure the `.zip` archive for your topic exists in `GDRIVE_ARCHIVE_PATH`.
    2.  Fill in the `TOPIC_IDS_FOR_OFFLINE_MODES` list with the desired topic ID(s).
    3.  Execute the cell containing the `run_notion_import(...)` function.

*   **How to verify it worked:**
    *   The notebook will log the creation of the page and the batches of blocks being sent.
    *   A new sub-page with the topic's title will appear under your `NOTION_PARENT_PAGE_ID` in Notion.

### Workflow 3: Resuming a Failed Import (Manual Recovery)

This is a special workflow for recovering from an error (e.g., a network failure, a manual `Ctrl+C` interruption).

**Scenario:** An import fails. The notebook output shows a message like this:

```
❌ HALTED: Import process stopped unexpectedly.
-> Last processed Rizzoma node ID was: 0_b_cjo2_co4lp
-> To resume, use this ID in the 'resume_single_topic_import' function.
```

*   **How to run the recovery:**
    1.  **Find the last processed node ID:** Copy the ID from the log message (`0_b_cjo2_co4lp` in the example).
    2.  **Find the partial Notion page:** Go to Notion. You will find an incomplete page that was created. Copy its ID from the URL.
    3.  **Execute the `resume_single_topic_import` function** with the information you collected:
        ```python
        resume_single_topic_import(
            topic_id='...',             # The ID of the topic that failed
            target_page_id='...',       # The ID of the incomplete Notion page
            start_at_node_id='...',     # The Rizzoma node ID from the error log
            prune_old_blips=True
        )
        ```
*   **Important Note (The "Flattened Resume" Approach):**
    This function prioritizes robustness over perfect structure. It will add **all remaining content to the root of the Notion page**. If the failure occurred inside a toggle that is now empty, you will need to perform one manual step:
    *   Go to the Notion page, select all the newly added blocks, and drag-and-drop them inside the empty toggle.
    This trade-off makes the resume process extremely reliable and prevents complex bugs.



# Environment setup

In [1]:
# --- 1. Install Dependencies ---
!pip install requests beautifulsoup4 tqdm notion-client tenacity -q

# --- 2. Python Standard & Third-Party Library Imports ---
import os
import json
import shutil
import time
import re
import mimetypes
from pathlib import Path
from urllib.parse import urlparse
from datetime import datetime
from contextlib import contextmanager

import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from google.colab import drive, userdata
import notion_client
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

print("✅ All required libraries installed and imported successfully.")

# --- 3. Mount Google Drive ---
drive.mount('/content/drive', force_remount=True)

✅ All required libraries installed and imported successfully.
Mounted at /content/drive


# Configuration

In [ ]:
# --- Rizzoma Credentials ---
# Your Rizzoma session cookie ID, retrieved from your browser.
CONNECT_SID = userdata.get('CONNECT_SID')

# --- Google Drive & Local Paths ---
# The path on your Google Drive where final ZIP archives will be stored/read from.
GDRIVE_ARCHIVE_PATH = '/content/drive/MyDrive/rizzoma_archive'
# The local temporary folder for processing files.
TEMP_WORK_DIR = 'rizzoma_export_temp'

# --- Notion API Configuration ---
# Your Notion Integration Token.
NOTION_API_KEY = userdata.get('NOTION_API_KEY')
# The ID of the parent page in Notion where new topic pages will be created.
NOTION_PARENT_PAGE_ID = ''

# --- Script Behavior ---
# If True, any Rizzoma line containing only "old" followed by a blip will be skipped during Notion import.
PRUNE_OLD_BLIPS = True
# Rate limiting delay in seconds to avoid overwhelming the Rizzoma API.
TIME_DELAY_S = 0.3

print("✅ Configuration loaded.")

✅ Configuration loaded.


# Toolkit

## Utilities

In [4]:
# --- Utility Functions ---
def sanitize_name(name):
    """Removes characters invalid for file or directory names."""
    return re.sub(r'[<>:\"/\\\\|?*]', '_', name).strip()

def get_unique_filepath(path):
    """Returns a unique file path by adding a number suffix if the path already exists."""
    if not path.exists(): return path
    parent, stem, ext = path.parent, path.stem, path.suffix
    i = 1
    while True:
        new_path = parent / f"{stem} ({i}){ext}"
        if not new_path.exists(): return new_path
        i += 1

def _prepare_topic_from_gdrive(topic_id):
    """
    Finds a topic ZIP on GDrive, copies and unpacks it to the local temp directory.
    Implements a simple cache: if the directory already exists locally, it skips the download.
    """
    unzip_dir = Path(TEMP_WORK_DIR) / f"topic__{topic_id}"
    if unzip_dir.exists():
        print(f"-> Found local cache for Topic ID {topic_id} at '{unzip_dir}'. Skipping download.")
        return unzip_dir

    Path(TEMP_WORK_DIR).mkdir(exist_ok=True)
    gdrive_path = Path(GDRIVE_ARCHIVE_PATH)
    if not gdrive_path.is_dir():
        print(f"❌ ERROR: Archive directory '{gdrive_path}' not found."); return None

    zip_files = list(gdrive_path.glob(f"*__{topic_id}.zip"))
    if not zip_files:
        print(f"-> WARNING: No archive found on Google Drive for Topic ID {topic_id}."); return None

    zip_path = zip_files[0]
    print(f"\n-> Processing archive: {zip_path.name}")
    shutil.copy(str(zip_path), TEMP_WORK_DIR)

    # Create a clean directory with a predictable name
    unzip_dir.mkdir()
    shutil.unpack_archive(str(Path(TEMP_WORK_DIR) / zip_path.name), str(unzip_dir), 'zip')

    (Path(TEMP_WORK_DIR) / zip_path.name).unlink()
    return unzip_dir

## Rizzoma

In [5]:
class RizzomaParser:
    """Parses Rizzoma JSON to build a map of all files."""
    def __init__(self, json_data):
        self.json_data = json_data
        self.file_map = {}
        # This will hold a reconstructed map for offline modes
        self.local_file_map = {}

    def build_file_map_from_api(self):
        self._find_files_recursive_api(self.json_data.get('nodes', []))
        return self.file_map

    def reconstruct_file_map_from_local(self, files_dir: Path):
        """Scans the local 'files' directory to rebuild a usable file map."""
        for node_dir in files_dir.iterdir():
            if node_dir.is_dir():
                for file_path in node_dir.iterdir():
                    # We create a simple map from filename to its full local path
                    self.local_file_map[file_path.name] = file_path
        return self.local_file_map

    def _find_files_recursive_api(self, nodes, current_reply_id=None):
        if not isinstance(nodes, list): return
        for node in nodes:
            reply_id = node.get('id') if node.get('type') == 'reply' else current_reply_id
            if node.get('type') == 'file' and 'url' in node and reply_id:
                self._add_file_to_map(node, reply_id)
                if 'preview' in node:
                    self._add_file_to_map(node, reply_id, is_preview=True)
            if 'nodes' in node:
                self._find_files_recursive_api(node.get('nodes', []), reply_id)

    def _add_file_to_map(self, node, reply_id, is_preview=False):
        url = node['preview'] if is_preview else node['url']
        name = node.get('name', Path(urlparse(url).path).name)
        if is_preview:
            p = Path(name)
            name = f"thumbnail_{p.stem}{p.suffix}" if p.suffix else f"thumbnail_{p.stem}"
        self.file_map[url] = {'node_id': reply_id, 'filename': name, 'final_name': None}

class HtmlModifier:
    """Modifies the original Rizzoma HTML to use local file links."""
    def __init__(self, html_path, file_map, all_topics_meta):
        self.path = html_path
        self.file_map = file_map
        self.all_topics_meta = all_topics_meta
        with open(html_path, 'r', encoding='utf-8') as f:
            self.soup = BeautifulSoup(f, 'html.parser')

    def process_and_save(self, output_path):
        self._rewrite_attachment_links()
        # Inter-topic link rewriting can be added here if needed
        output_path.write_text(str(self.soup), encoding='utf-8')

    def _rewrite_attachment_links(self):
        tags = self.soup.select('a[href*="rizzoma.com/r/files/"], img[src*="rizzoma.com/r/files/"]')
        for tag in tags:
            attr = 'href' if tag.has_attr('href') else 'src'
            if tag[attr] in self.file_map:
                info = self.file_map[tag[attr]]
                tag[attr] = f"files/{info['node_id']}/{info['final_name']}"

## Notion

In [6]:
# A set of all file extensions officially supported by the Notion API for fast lookups.
NOTION_SUPPORTED_EXTENSIONS = {
    ".aac", ".adts", ".mid", ".midi", ".mp3", ".mpga", ".m4a", ".m4b", ".mp4", ".oga", ".ogg", ".wav", ".wma",
    ".pdf", ".txt", ".json", ".doc", ".dot", ".docx", ".dotx", ".xls", ".xlt", ".xla", ".xlsx", ".xltx", ".ppt", ".pot", ".pps", ".ppa", ".pptx", ".potx",
    ".gif", ".heic", ".jpeg", ".jpg", ".png", ".svg", ".tif", ".tiff", ".webp", ".ico",
    ".amv", ".asf", ".wmv", ".avi", ".f4v", ".flv", ".gifv", ".m4v", ".mkv", ".webm", ".mov", ".qt", ".mpeg"
}

# Maps Rizzoma's specific background color hex codes to Notion's named background colors.
RIZZOMA_TO_NOTION_BG_COLOR_MAP = {
    '#B5EB5E': 'green_background',
    '#FFE066': 'yellow_background',
    '#8AE5D6': 'blue_background',
    '#AAB2F2': 'purple_background',
    '#DAB6F2': 'pink_background',
    '#F2A79D': 'red_background'
}

# Define which Notion API errors are considered temporary and should trigger a retry.
def is_retryable_notion_error(e: Exception) -> bool:
    """Return True if the exception is a Notion API error worth retrying."""
    if not isinstance(e, notion_client.errors.APIResponseError):
        return False
    # 500, 502, 503, 504 are server-side errors. 408 is a timeout. 429 is rate limiting.
    return e.status in [408, 429, 500, 502, 503, 504]


class NotionImporter:
    """
    Handles the import of a Rizzoma topic into Notion with resilience features.
    """
    def __init__(self, api_key, unzip_dir, file_manifest, prune_old_blips=False):
        self.notion = notion_client.Client(auth=api_key)
        self.unzip_dir = Path(unzip_dir)
        self.file_manifest = file_manifest
        self.prune_old_blips = prune_old_blips
        self.current_node_id = None
        self.state_file_path = self.unzip_dir / "_import_state.json"
        self.resuming_found_start_node = False


    def import_topic(self, rizzoma_nodes, topic_title, parent_page_id, start_at_rizzoma_node_id=None, append_mode=False):
        """
        Main import orchestrator. Creates a new page if append_mode is False,
        otherwise appends to the existing page identified by parent_page_id.
        """
        page_id = parent_page_id
        # --- Initialize the shared state for this import run ---
        self.resuming_found_start_node = (start_at_rizzoma_node_id is None)

        try:
            if not append_mode:
                print(f"\n-> Creating new Notion page for '{topic_title}'...")
                page_id = self.create_page(topic_title, parent_page_id)
            else:
                print(f"\n-> Appending to existing Notion page: {page_id}")

            if page_id:
                print("-> Starting resilient block creation...")
                self._create_blocks_iteratively(rizzoma_nodes, page_id, start_at_rizzoma_node_id)

            return page_id
        except (Exception, KeyboardInterrupt) as e:
            print(f"\n❌ HALTED: Import process stopped unexpectedly.")
            if self.current_node_id:
                print(f"   -> Last processed Rizzoma node ID was: {self.current_node_id}")
                print(f"   -> To resume, use this ID in the 'resume_single_topic_import' function.")
            raise

    def create_page(self, topic_title, parent_id):
        """Creates the main Notion page for the topic."""
        try:
            new_page = self.notion.pages.create(
                parent={"page_id": parent_id},
                properties={"title": [{"type": "text", "text": {"content": topic_title}}]}
            )
            return new_page['id']
        except Exception as e:
            print(f"  -> ❌ An API error occurred during page creation: {e}")
            return None

    @retry(
        wait=wait_exponential(multiplier=1, min=2, max=60), # Waits 2s, 4s, 8s, ...
        stop=stop_after_attempt(5),
        retry=retry_if_exception_type(notion_client.errors.APIResponseError)
    )

    def _append_blocks_resiliently(self, parent_id, blocks):
        """
        A resilient wrapper for the Notion API append call. This is the only
        function that should directly call the blocks.children.append endpoint.
        """
        if not blocks:
            return None

        return self.notion.blocks.children.append(parent_id, children=blocks)

    def _update_state_file(self, node_id):
        """Saves the last successfully processed node ID to a state file."""
        self.current_node_id = node_id
        with open(self.state_file_path, 'w') as f:
            json.dump({'last_successful_node_id': node_id}, f)

    def _flush_blocks_batch(self, blocks_batch, parent_block_id):
        """Helper function to send a batch of simple blocks and reset the batch."""
        if not blocks_batch:
            return []

        print(f"    -> Sending a batch of {len(blocks_batch)} simple blocks...")
        try:
            self._append_blocks_resiliently(parent_block_id, blocks_batch)
        except Exception as e:
            print(f"    -> ❌ ERROR sending batch: {e}")

        return []

    def _create_blocks_iteratively(self, rizzoma_nodes, parent_block_id, start_at_id=None):
        """
        Processes Rizzoma nodes with a corrected, stateful resumability logic
        that prevents duplication and infinite loops.
        """
        if not rizzoma_nodes: return

        blocks_batch = []

        for node in rizzoma_nodes:
            node_id = node.get('id')
            if node_id:
                self.current_node_id = node_id

            if not self.resuming_found_start_node:
                if node_id == start_at_id:
                    print(f"✅ Found resume point at node ID: {start_at_id}. Resuming processing.")
                    self.resuming_found_start_node = True
                else:
                    if 'nodes' in node:
                        self._create_blocks_iteratively(node.get('nodes', []), parent_block_id, start_at_id)
                    continue

            node_type = node.get('type')

            if node_type in ['paragraph', 'listItem']:
                children_nodes = node.get('nodes', [])
                if not children_nodes and node_type == 'paragraph':
                    blocks_batch.append({"type": "paragraph", "paragraph": {"rich_text": []}})
                else:
                    text_nodes_buffer = []
                    for child_node in children_nodes:
                        child_type = child_node.get('type')
                        if child_type == 'text':
                            text_nodes_buffer.append(child_node)
                        elif child_type in ['file', 'attachment']:
                            if text_nodes_buffer:
                                rich_text_content = self._convert_nodes_to_rich_text(text_nodes_buffer)
                                block_type = "paragraph" if node_type == 'paragraph' else "bulleted_list_item"
                                blocks_batch.append({"type": block_type, block_type: {"rich_text": rich_text_content}})
                                text_nodes_buffer = []
                            if child_type == 'file':
                                file_block = self._process_file_node(child_node)
                                if file_block: blocks_batch.append(file_block)
                            elif child_type == 'attachment':
                                url = child_node.get('url')
                                if url: blocks_batch.append({"type": "image", "image": {"type": "external", "external": {"url": url}}})
                        elif child_type == 'thread':
                            blocks_batch = self._flush_blocks_batch(blocks_batch, parent_block_id)
                            toggle_title = self._convert_nodes_to_rich_text(text_nodes_buffer)
                            text_nodes_buffer = []
                            text_for_pruning = "".join([rt.get('plain_text', '') for rt in toggle_title])
                            if self.prune_old_blips and text_for_pruning.strip().lower() == 'old':
                                print("    -> Pruning 'old' archive blip. Skipping node.")
                                continue
                            print(f"    -> Creating toggle '{text_for_pruning[:40]}...' and recursing...")
                            new_toggle_block = {"type": "toggle", "toggle": {"rich_text": toggle_title}}
                            response = self._append_blocks_resiliently(parent_block_id, [new_toggle_block])
                            new_toggle_id = response['results'][0]['id']
                            self._create_blocks_iteratively(child_node.get('nodes', []), new_toggle_id, None)
                    if text_nodes_buffer:
                        rich_text_content = self._convert_nodes_to_rich_text(text_nodes_buffer)
                        block_type = "paragraph" if node_type == 'paragraph' else "bulleted_list_item"
                        blocks_batch.append({"type": block_type, block_type: {"rich_text": rich_text_content}})

            elif node_type in ['reply', 'thread', 'list']:
                blocks_batch = self._flush_blocks_batch(blocks_batch, parent_block_id)
                self._create_blocks_iteratively(node.get('nodes', []), parent_block_id, None)

            elif node_type in ['file', 'attachment']:
                block = self._process_file_node(node) if node_type == 'file' else {"type": "image", "image": {"type": "external", "external": {"url": node.get('url')}}}
                if block: blocks_batch.append(block)

            if node_id:
                self._update_state_file(node_id)

        blocks_batch = self._flush_blocks_batch(blocks_batch, parent_block_id)

    def _convert_nodes_to_rich_text(self, rizzoma_nodes, inherited_url=None, inherited_annotations={}):
        """
        Recursively converts Rizzoma nodes into a flat list of Notion rich text objects,
        correctly handling nested formatting, links, and colors.
        """
        rich_text_array = []
        for node in rizzoma_nodes:
            if node.get('type') != 'text': continue

            # Inherit properties from parent nodes
            annotations = inherited_annotations.copy()
            link_url = inherited_url or node.get('url')

            # Apply current node's formatting
            if node.get('bold'): annotations['bold'] = True
            if node.get('italic'): annotations['italic'] = True
            if node.get('underlined'): annotations['underline'] = True
            if node.get('struckthrough'): annotations['strikethrough'] = True
            if node.get('bgColor') in RIZZOMA_TO_NOTION_BG_COLOR_MAP:
                annotations['color'] = RIZZOMA_TO_NOTION_BG_COLOR_MAP[node.get('bgColor')]

            # Create the rich text object if there's text content
            if 'value' in node:
                content = node.get('value')
                if content:
                    text_object = {"content": content}
                    if link_url:
                        text_object['link'] = {"url": link_url}

                    rt_object = {
                        "type": "text",
                        "text": text_object,
                        "annotations": annotations,
                        "plain_text": content
                    }
                    rich_text_array.append(rt_object)

            # Recurse for nested nodes, passing down the collected properties
            if 'nodes' in node:
                rich_text_array.extend(self._convert_nodes_to_rich_text(node['nodes'], link_url, annotations))

        return rich_text_array

    @contextmanager
    def _managed_file_for_upload(self, original_path: Path):
        """
        Context manager to provide a Notion-compatible file path. It creates a temporary
        copy with a .txt extension if the original file type is unsupported.
        """
        temp_path = None
        file_ext = original_path.suffix.lower()

        if file_ext in NOTION_SUPPORTED_EXTENSIONS:
            yield original_path
        else:
            print(f"    -> Unsupported extension '{file_ext}'. Creating temporary '.txt' version.")
            temp_path = original_path.parent / (original_path.name + ".txt")
            try:
                shutil.copy2(original_path, temp_path)
                yield temp_path
            finally:
                if temp_path and os.path.exists(temp_path):
                    os.remove(temp_path)
                    print(f"    -> Cleaned up temporary file: {temp_path.name}")

    def _upload_file_and_get_id(self, local_file_path):
        """
        Handles the 2-step file upload process (declare intent, then send data).
        Returns the file_upload_id on success.
        """
        if not os.path.exists(local_file_path):
            print(f"    -> ❌ Local file not found: {local_file_path}")
            return None
        try:
            with self._managed_file_for_upload(Path(local_file_path)) as path_to_upload:
                file_name_for_notion = os.path.basename(path_to_upload)

                # Step A: Declare the upload
                create_response = self.notion.file_uploads.create(mode="single_part", filename=file_name_for_notion)
                file_upload_id = create_response["id"]

                # Step B: Send the binary data
                with open(path_to_upload, "rb") as f:
                    send_response = self.notion.file_uploads.send(file_upload_id=file_upload_id, file=f)

                if send_response["status"] != "uploaded":
                    raise Exception(f"Upload failed with status: {send_response['status']}")

                print(f"    -> ✅ Successfully uploaded {file_name_for_notion}")
                return file_upload_id
        except Exception as e:
            print(f"    -> ❌ Error during file upload for {local_file_path}: {e}")
            return None

    def _create_rich_text_from_node(self, node):
        """Helper to create a rich text object from a 'text' node with annotations."""
        if node.get('type') != 'text': return []
        content = node.get('value', '')
        annotations = {
            'bold': node.get('bold', False), 'italic': node.get('italic', False),
            'strikethrough': node.get('struckthrough', False), 'underline': node.get('underlined', False)
        }
        return [{"type": "text", "text": {"content": content}, "annotations": annotations}] if content else []

    def _process_file_node(self, node):
        """
        Helper to create a file or image block dictionary. It now handles both
        local files that need uploading and external images that are linked directly.
        """
        file_url = node.get('url')

        if file_url in self.file_manifest:
            # --- CASE 1: LOCAL FILE (Hosted on Rizzoma) ---
            # The URL is in our manifest, so we have a local copy to upload.
            file_info = self.file_manifest[file_url]
            local_path = self.unzip_dir / "files" / file_info['node_id'] / file_info['final_name']

            print(f"-> Processing local file: {file_info['final_name']}...")

            file_id = self._upload_file_and_get_id(local_path)
            if file_id:
                is_image = 'preview' in node
                block_type = "image" if is_image else "file"
                return {
                    "type": block_type,
                    block_type: {
                        "type": "file_upload",
                        "file_upload": {"id": file_id}
                    }
                }

        elif file_url and file_url.startswith('http'):
            # --- CASE 2: EXTERNAL IMAGE (Not hosted on Rizzoma) ---
            # The URL is not in our manifest, so we treat it as an external link.
            # This typically applies only to images.
            print(f"-> Processing as external image: {file_url}")
            return {
                "type": "image",
                "image": {
                    "type": "external",
                    "external": {
                        "url": file_url
                    }
                }
            }

        else:
            # --- FALLBACK CASE ---
            # The URL is missing or invalid.
            print(f"-> ⚠️ File URL missing or not found in manifest. Creating placeholder for: {node.get('name', 'unknown file')}")
            return {
                "type": "callout",
                "callout": {
                    "rich_text": [{"type": "text", "text": {"content": f"File URL missing or invalid: {node.get('name', 'file')}"}}],
                    "icon": {"emoji": "❓"}
                }
            }

        return None

# Orchestration workflows



## run_acquisition

In [7]:
def run_acquisition(topic_ids):
    """
    ACQUISITION MODULE: Creates a complete, immutable archive for each topic ID.
    This is the ONLY function that communicates with the Rizzoma API.
    """
    print(f"\n🚀 STARTING ACQUISITION MODE for {len(topic_ids)} topic(s)...\n")
    Path(TEMP_WORK_DIR).mkdir(exist_ok=True)

    for topic_id in tqdm(topic_ids, desc="Overall Acquisition Progress", unit="topic"):
        topic_title = f"Untitled_{topic_id}"
        try:
            print(f"\nProcessing Topic ID: {topic_id}")

            # 1. Download Source Data
            print("-> Downloading topic data (JSON and HTML)...")
            response_json = requests.get(f'https://rizzoma.com/api/export/1/{topic_id}/json/', headers={'Cookie': f'connect.sid={CONNECT_SID}'})
            response_json.raise_for_status()
            json_data = response_json.json()
            topic_title = json_data.get('title', topic_title)

            response_html = requests.get(f'https://rizzoma.com/api/export/1/{topic_id}/html/', headers={'Cookie': f'connect.sid={CONNECT_SID}'})
            response_html.raise_for_status()

            # 2. Setup Local Directory
            sanitized_title = sanitize_name(topic_title)
            topic_dir = Path(TEMP_WORK_DIR) / sanitized_title
            topic_dir.mkdir(exist_ok=True)
            files_dir = topic_dir / "files"; files_dir.mkdir(exist_ok=True)

            (topic_dir / f"{sanitized_title}.json").write_bytes(response_json.content)
            original_html_path = topic_dir / f"{sanitized_title}_original_api.html"
            original_html_path.write_bytes(response_html.content)

            # 3. Parse JSON, Download Attachments, and Handle Errors
            print("-> Parsing JSON to find and download all attachments...")
            parser = RizzomaParser(json_data)
            file_map = parser.build_file_map_from_api()

            # --- NEW: Initialize tracking variables for the log ---
            successful_downloads = 0
            failed_downloads = []

            if file_map:
                for url, info in tqdm(file_map.items(), desc=f"Downloading {len(file_map)} files", leave=False):
                    try:
                        node_dir = files_dir / info['node_id']; node_dir.mkdir(exist_ok=True)

                        res = requests.get(url, headers={'Cookie': f'connect.sid={CONNECT_SID}'}, stream=True)
                        res.raise_for_status() # This will raise an HTTPError for 4xx/5xx responses

                        p = Path(info['filename']); ext = p.suffix or mimetypes.guess_extension(res.headers.get('content-type')) or ''
                        unique_path = get_unique_filepath(node_dir / f"{p.stem}{ext}")

                        with open(unique_path, 'wb') as f:
                            shutil.copyfileobj(res.raw, f)

                        info['final_name'] = unique_path.name
                        successful_downloads += 1
                        time.sleep(TIME_DELAY_S)

                    except requests.exceptions.HTTPError as e:
                        # --- NEW: Catch HTTP errors (like 404) without crashing ---
                        print(f"-> WARNING: HTTP {e.response.status_code} for {url}. Skipping file.")
                        failed_downloads.append({'url': url, 'filename': info['filename'], 'error': str(e)})
                        info['final_name'] = None # Mark as failed in the map
                    except Exception as e:
                        # Catch other potential errors (e.g., network issues)
                        print(f"-> WARNING: An unexpected error occurred for {url}. Skipping file. Error: {e}")
                        failed_downloads.append({'url': url, 'filename': info['filename'], 'error': str(e)})
                        info['final_name'] = None

            # 4. Create File Manifest and Download Log
            with open(topic_dir / "_file_manifest.json", 'w', encoding='utf-8') as f:
                json.dump(file_map, f, indent=2)

            print("-> Generating download log...")
            log_content = [
                f"# Download Log for Topic: {topic_title}\n\n",
                "**Summary:**\n",
                f"- Files detected in JSON: {len(file_map)}\n",
                f"- Files successfully downloaded: {successful_downloads}\n",
                f"- Files failed to download: {len(failed_downloads)}\n"
            ]
            if failed_downloads:
                log_content.append("\n---\n\n**Failed Files:**\n\n")
                for failed in failed_downloads:
                    log_content.append(f"- **File:** `{failed['filename']}`\n")
                    log_content.append(f"  - **URL:** {failed['url']}\n")
                    log_content.append(f"  - **Error:** {failed['error']}\n")
            (topic_dir / "download_log.md").write_text("".join(log_content), encoding='utf-8')

            # 5. Generate Other Deliverables
            print("-> Generating HTML version with local file links...")
            modified_html_path = topic_dir / f"{sanitized_title}_local_links.html"
            HtmlModifier(original_html_path, file_map, {}).process_and_save(modified_html_path)

            # 6. Package and Upload to Google Drive
            zip_name = f"{sanitized_title}__{topic_id}"
            archive_path = Path(shutil.make_archive(zip_name, 'zip', str(topic_dir)))
            gdrive_path = Path(GDRIVE_ARCHIVE_PATH); gdrive_path.mkdir(parents=True, exist_ok=True)
            print(f"-> Uploading '{archive_path.name}' to Google Drive...")
            shutil.move(str(archive_path), str(gdrive_path / archive_path.name))

        except Exception as e:
            # This outer try/except catches fatal errors (e.g., can't download the main JSON)
            print(f"\n❌ FATAL ERROR processing topic {topic_id} ('{topic_title}'). Skipping. Details: {e}")

    print("\n✅ ACQUISITION MODE FINISHED.")

## run_notion_import

In [8]:
def run_notion_import(topic_ids, prune_old_blips=False):
    """
    NOTION IMPORT MODE: Starts a new import for a list of topics, creating a new
    Notion page for each one.
    """
    print(f"\n🚀 STARTING NEW NOTION IMPORT for {len(topic_ids)} topic(s)...\n")
    if not NOTION_API_KEY or 'REPLACE_WITH' in NOTION_API_KEY:
        print("❌ ERROR: Notion API Key is not configured. Aborting."); return

    for topic_id in tqdm(topic_ids, desc="Importing to Notion"):
        unzip_dir = _prepare_topic_from_gdrive(topic_id)
        if not unzip_dir: continue

        state_file_path = unzip_dir / "_import_state.json"
        if state_file_path.exists():
            print(f"-> Found and removed old state file for fresh import.")
            state_file_path.unlink()

        try:
            print(f"-> Searching for topic data in '{unzip_dir}'...")
            json_files = [p for p in unzip_dir.glob('*.json') if not p.name.startswith('_')]
            manifest_path = unzip_dir / "_file_manifest.json"

            if not json_files or not manifest_path.exists():
                print(" -> ❌ ERROR: Main JSON or file manifest is missing. Skipping."); continue

            json_path = json_files[0]

            with open(json_path, 'r', encoding='utf-8') as f: json_data = json.load(f)
            with open(manifest_path, 'r', encoding='utf-8') as f: file_manifest = json.load(f)
            print(f"-> Successfully loaded '{json_path.name}' and its file manifest.")

            importer = NotionImporter(
                NOTION_API_KEY, unzip_dir, file_manifest, prune_old_blips
            )

            importer.import_topic(
                rizzoma_nodes=json_data.get('nodes', []),
                topic_title=json_data.get('title', unzip_dir.name),
                parent_page_id=NOTION_PARENT_PAGE_ID,
                append_mode=False
            )

            print(f"-> Finished processing for topic {topic_id}.")

        except (Exception, KeyboardInterrupt):
            print(f"\n-> Process for topic {topic_id} was interrupted. To resume, use 'resume_single_topic_import'.")
            continue

In [9]:
def resume_single_topic_import(topic_id, target_page_id, start_at_node_id, prune_old_blips=False):
    """
    RESUME MODE: Resumes a failed import for a SINGLE topic by appending all
    remaining content to the ROOT of an EXISTING Notion page.

    The user is responsible for manually moving the newly created blocks into their
    final nested position (e.g., inside an empty toggle) if necessary.

    Args:
        topic_id (str): The Rizzoma Topic ID to process.
        target_page_id (str): The ID of the existing Notion page to append content to.
        start_at_node_id (str): The Rizzoma node ID from which to resume processing.
        prune_old_blips (bool): Whether to prune 'old' blips.
    """
    print(f"\n🚀 RESUMING IMPORT for topic '{topic_id}' at the root of page '{target_page_id}'...\n")
    if not all([topic_id, target_page_id, start_at_node_id]):
        print("❌ ERROR: 'topic_id', 'target_page_id', and 'start_at_node_id' are all required."); return

    unzip_dir = _prepare_topic_from_gdrive(topic_id)
    if not unzip_dir: return

    try:
        json_path = next((p for p in unzip_dir.glob('*.json') if not p.name.startswith('_')), None)
        manifest_path = unzip_dir / "_file_manifest.json"
        if not json_path or not manifest_path.exists():
            print(" -> ❌ ERROR: Main JSON or file manifest is missing. Skipping."); return

        with open(json_path, 'r', encoding='utf-8') as f: json_data = json.load(f)
        with open(manifest_path, 'r', encoding='utf-8') as f: file_manifest = json.load(f)
        print(f"-> Successfully loaded '{json_path.name}' and its file manifest.")

        importer = NotionImporter(
            NOTION_API_KEY, unzip_dir, file_manifest, prune_old_blips
        )

        # We always append to the root of the target page. No complex targeting.
        importer.import_topic(
            rizzoma_nodes=json_data.get('nodes', []),
            topic_title=json_data.get('title', unzip_dir.name),
            parent_page_id=target_page_id,
            start_at_rizzoma_node_id=start_at_node_id,
            append_mode=True
        )

        state_file_path = unzip_dir / "_import_state.json"
        if state_file_path.exists():
            print("-> Resume successful. Removing state file.")
            state_file_path.unlink()

        print(f"-> Finished resuming for topic {topic_id}.")

    except (Exception, KeyboardInterrupt):
        print(f"\n-> Resume process for topic {topic_id} was interrupted. Check logs for the new resume point.")

# Run Rizzoma export

In [10]:
# This will connect to the Rizzoma API and create fresh archives on Google Drive.
run_acquisition(topic_ids=[
    '366d5a79a54be17288daab770d4833d9',  #rizzoma export full test
            ]
)


🚀 STARTING ACQUISITION MODE for 1 topic(s)...



Overall Acquisition Progress:   0%|          | 0/1 [00:00<?, ?topic/s]


Processing Topic ID: 366d5a79a54be17288daab770d4833d9
-> Downloading topic data (JSON and HTML)...
-> Parsing JSON to find and download all attachments...


-> Generating download log...
-> Generating HTML version with local file links...
-> Uploading 'rizzoma export full test__366d5a79a54be17288daab770d4833d9.zip' to Google Drive...

✅ ACQUISITION MODE FINISHED.


# Run Notion import

In [15]:
# This will import the specified topics from your GDrive archives into Notion.
run_notion_import(topic_ids=[
    '366d5a79a54be17288daab770d4833d9',  #rizzoma export full test
    ],
    prune_old_blips=True
)


🚀 STARTING NEW NOTION IMPORT for 1 topic(s)...



Importing to Notion:   0%|          | 0/1 [00:00<?, ?it/s]

-> Found local cache for Topic ID 366d5a79a54be17288daab770d4833d9 at 'rizzoma_export_temp/topic__366d5a79a54be17288daab770d4833d9'. Skipping download.
-> Searching for topic data in 'rizzoma_export_temp/topic__366d5a79a54be17288daab770d4833d9'...
-> Successfully loaded 'rizzoma export full test.json' and its file manifest.

-> Creating new Notion page for 'rizzoma export full test'...
-> Starting resilient block creation...
    -> Sending a batch of 2 simple blocks...
    -> Creating toggle 'images ...' and recursing...
-> Processing local file: image.png...
    -> ✅ Successfully uploaded image.png
-> Processing local file: image (1).png...
    -> ✅ Successfully uploaded image (1).png
-> Processing local file: image_custom_name.png...
    -> ✅ Successfully uploaded image_custom_name.png
-> Processing local file: image (2).png...
    -> ✅ Successfully uploaded image (2).png
    -> Sending a batch of 10 simple blocks...
    -> Creating toggle 'files ...' and recursing...
-> Processing l

In [17]:
"""
RESUME MODE: Resumes a failed import for a SINGLE topic.

Args:
    topic_id (str): The Rizzoma Topic ID to process.
    start_at_node_id (str): The Rizzoma node ID from which to resume processing.
    target_page_id (str): The ID of the Notion page containing the partial import.
    prune_old_blips (bool): Whether to prune 'old' blips.
"""


resume_single_topic_import(
    topic_id='366d5a79a54be17288daab770d4833d9',
    start_at_node_id= '0_b_cjo2_co4o8', #resume at "nested_blips" node
    target_page_id='2ab0e93a826b819a800fdbbe6964a633', #you need to get the id of the page created on Notion for the page you want to resume
    prune_old_blips=True
)


🚀 RESUMING IMPORT for topic '366d5a79a54be17288daab770d4833d9' at the root of page '2ab0e93a826b819a800fdbbe6964a633'...

-> Found local cache for Topic ID 366d5a79a54be17288daab770d4833d9 at 'rizzoma_export_temp/topic__366d5a79a54be17288daab770d4833d9'. Skipping download.
-> Successfully loaded 'rizzoma export full test.json' and its file manifest.

-> Appending to existing Notion page: 2ab0e93a826b819a800fdbbe6964a633
-> Starting resilient block creation...
✅ Found resume point at node ID: 0_b_cjo2_co4o8. Resuming processing.
    -> Creating toggle 'no_bullet_blip ...' and recursing...
    -> Sending a batch of 1 simple blocks...
    -> Sending a batch of 1 simple blocks...
    -> Creating toggle 'bullet_blip ...' and recursing...
    -> Sending a batch of 1 simple blocks...
    -> Creating toggle 'multi blip same line: blip_1 ...' and recursing...
    -> Sending a batch of 1 simple blocks...
    -> Creating toggle ' blip_2 ...' and recursing...
    -> Sending a batch of 1 simple bl